In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# credit card Fraud Detection
# Veri seti:
* Bu proje : kredi kartının kullaıldığı her yerde bulunan kredi kartı dolandırıcılık tespiti yapılmasını amaçlamaktadır.BAnkalarda,e-commerce(e ticaret siteleri),ve kredi kartının geçtiği her yerde karşımıza çıkabilir.
* amaç :kredi kartı dolandırıclık tespiti yaparak tahminde bulunmak ve tespit etmektir.
* sınıflandırma projesidir.machine learning kullanılacaktır.
* v1 -v29 feature değişkenleri standardlaştırlımış.
* işem yapılan yer, lokasyon,site gibi değerler bunlar gizlenmiştir.
* bizden beklenen bu v columnlarından önemli olanları bulup frauda sebep verenleri bulmaktır.

In [ ]:
df=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [ ]:
df #id column primary keydir v1-v29 columlartı gizlenmiş feature dır.yani bunlar müşteriye ait herhangi bir özelliktir.

In [ ]:
# amount=ücret miktarıdır. class 1 ise sahtekarlık vardır. 0 ise yoktur demektir.

In [ ]:
df.Class.value_counts()

In [ ]:
print(f"%{df.Class.value_counts()[0]/len(df) *100} fraud olmayan")
print(f"%{df.Class.value_counts()[1]/len(df)*100} fraud")

In [ ]:
df.isnull().sum().max()

In [ ]:
df.columns

In [ ]:
sns.countplot(df,x="Class") #hemen hemen 1/0 eşit gibi görünüyor.

In [ ]:
df.describe()

In [ ]:
#we see that we have sclaed feature but we have to scale all feature (Time ,Amount) to create model.We have used Robust
#scaler because this choice more resilince for outlier values

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df.Time = scaler.fit_transform(df.Time.values.reshape(-1,1))
df.Amount = scaler.fit_transform(df.Amount.values.reshape(-1,1))

In [ ]:
df

# undersample

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)
y = df['Class']

Xtrain, Xtest, ytrain,  ytest = train_test_split(X, y, test_size=0.2, random_state=818)

df = df.sample(frac=1)
fraud = df.loc[df.Class == 1]
non_fraud = df.loc[df.Class == 0][:492]

distributed_df = pd.concat([fraud, non_fraud])

new_df = distributed_df.sample(frac=1, random_state=818)

new_df.head()

sns.countplot(new_df,x="Class")

In [ ]:
relationship = new_df.corr()
sns.heatmap(relationship)

* 16-17-18 features have high relationship eventually that can be more re

In [ ]:
from scipy.stats import norm
new_df['V14'].loc[new_df['Class'] == 1].values
new_df['V12'].loc[new_df['Class'] == 1].values
new_df['V17'].loc[new_df['Class'] == 1].values
new_df['V18'].loc[new_df['Class'] == 1].values

In [ ]:
fig, axes = plt.subplots(1,3 ,figsize=(20,4))
axes[0].set_title("v11 class comparisons")
axes[1].set_title("v2 class comparisons ")
axes[2].set_title("v4 class comparisons")
sns.boxplot(x="Class", y="V11", data=new_df,   ax=axes[0])
sns.boxplot(x="Class", y="V2", data=new_df,   ax=axes[1])
sns.boxplot(x="Class", y="V4", data=new_df,   ax=axes[2])

# Outlier for undersample technique

In [ ]:
plt.figure(figsize=(20,6))
sns.boxplot(data = new_df)

In [ ]:
def outlier_zscore(data,threshold = 3):
    global outliers,zscore
    outliers = []
    zscore = []
    
    mean = np.mean(data)
    std = np.std(data)
    for i in data:
        z_score= (i - mean)/std 
        zscore.append(z_score)
        if np.abs(z_score) > threshold:
            outliers.append(i)
    cleaned_data = [x for x in data if x not in outliers]
    
    print("Total number of outliers are", len(outliers))
    
    return cleaned_data

In [ ]:
for i in df.columns:
    print (i)
    outlier_zscore(new_df[i])

In [ ]:
feature_list = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

def z_score_method(df,n,features):
    """
    Takes a dataframe df of features and returns an index list corresponding to the observations 
    containing more than n outliers according to the z-score method.
    """
    outlier_list = []
    
    for column in features:
        data_mean = df[column].mean()
        data_std = df[column].std()
        threshold = 3
        
        z_score = abs( (df[column] - data_mean)/data_std )
     
        outlier_list_column =  df[z_score > threshold].index
        
        outlier_list.extend(outlier_list_column)
        
    outlier_list = Counter(outlier_list)        
    multiple_outliers = list( k for k, v in outlier_list.items() if v > n )
    
    df1 = df[z_score > threshold]
    print('Total number of outliers is:', df1.shape[0])
    
    return multiple_outliers
# detecting outliers
#Outliers_z_score = z_score_method(new_df,1,feature_list)

# dropping outliers
#new_df = new_df.drop(Outliers_z_score, axis = 0).reset_index(drop=True)

In [ ]:
def outlier_thresholds(dataframe, col_names, q1=0.25, q3=0.75):
    limits = {}
    for col_name in col_names:
        quartile1 = dataframe[col_name].quantile(q1)
        quartile3 = dataframe[col_name].quantile(q3)
        interquantile_range = quartile3 - quartile1
        up_limit = quartile3 + 1.5 * interquantile_range
        low_limit = quartile1 - 1.5 * interquantile_range
        limits[col_name] = (low_limit, up_limit)
    return limits

def replace_with_thresholds(dataframe, variable, low_limit, up_limit):
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def replace_outliers(dataframe, col_names, q1=0.25, q3=0.75):
    limits = outlier_thresholds(dataframe, col_names, q1, q3)
    for col_name in col_names:
        low_limit, up_limit = limits[col_name]
        dataframe_copy = dataframe.copy()  # Orjinal veriyi değiştirmemek için bir kopya alıyoruz.
        outliers = dataframe_copy[(dataframe_copy[col_name] > up_limit) | (dataframe_copy[col_name] < low_limit)]
        if not outliers.empty:
            replace_with_thresholds(dataframe, col_name, low_limit, up_limit)
    return dataframe

def check_outlier(dataframe, col_names, q1=0.25, q3=0.75):
    outlier_dict = {}
    for col_name in col_names:
        low_limit, up_limit = outlier_thresholds(dataframe, [col_name], q1, q3)[col_name]
        if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
            outlier_dict[col_name] = True
        else:
            outlier_dict[col_name] = False
    return outlier_dict

In [ ]:
check_outlier(new_df,feature_list)

In [ ]:
new_df = replace_outliers(new_df,feature_list)

In [ ]:
check_outlier(new_df,feature_list)

In [ ]:
fig, axes = plt.subplots(1,3 ,figsize=(20,4))
axes[0].set_title("v11 class comparisons")
axes[1].set_title("v2 class comparisons ")
axes[2].set_title("v4 class comparisons")
sns.boxplot(x="Class", y="V11", data=new_df,   ax=axes[0])
sns.boxplot(x="Class", y="V2", data=new_df,   ax=axes[1])
sns.boxplot(x="Class", y="V4", data=new_df,   ax=axes[2])

In [ ]:
X = new_df.drop("Class",axis=1)
y = new_df["Class"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=818)
# Turn the values into an array for classification algorithms.
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold,cross_val_score,GridSearchCV,cross_val_predict

In [ ]:
for key, classifier in classifiers.items():
    classifier.fit(x_train, y_train)
    training_score = cross_val_score(classifier, x_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

In [ ]:
log_reg_params = {"penalty": ['l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

grid_log_reg = GridSearchCV(LogisticRegression(max_iter=1000), log_reg_params)
grid_log_reg.fit(x_train, y_train)
log_reg = grid_log_reg.best_estimator_

knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params)
grid_knears.fit(x_train, y_train)
# KNears best estimator
knears_neighbors = grid_knears.best_estimator_

# Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
grid_svc = GridSearchCV(SVC(), svc_params)
grid_svc.fit(x_train, y_train)

# SVC best estimator
svc = grid_svc.best_estimator_

# DecisionTree Classifier
tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)), 
              "min_samples_leaf": list(range(5,7,1))}
grid_tree = GridSearchCV(DecisionTreeClassifier(), tree_params)
grid_tree.fit(x_train, y_train)

# tree best estimator
tree_clf = grid_tree.best_estimator_

In [ ]:
log_reg_score = cross_val_score(log_reg, x_train, y_train, cv=5)
print('Logistic Regression Cross Validation Score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')


knears_score = cross_val_score(knears_neighbors, x_train, y_train, cv=5)
print('Knears Neighbors Cross Validation Score', round(knears_score.mean() * 100, 2).astype(str) + '%')

svc_score = cross_val_score(svc, x_train, y_train, cv=5)
print('Support Vector Classifier Cross Validation Score', round(svc_score.mean() * 100, 2).astype(str) + '%')

tree_score = cross_val_score(tree_clf, x_train, y_train, cv=5)
print('DecisionTree Classifier Cross Validation Score', round(tree_score.mean() * 100, 2).astype(str) + '%')

In [ ]:
log_reg_pred = cross_val_predict(log_reg, x_train, y_train, cv=5,
                             method="decision_function")

knears_pred = cross_val_predict(knears_neighbors, x_train, y_train, cv=5)

svc_pred = cross_val_predict(svc, x_train, y_train, cv=5,
                             method="decision_function")

tree_pred = cross_val_predict(tree_clf, x_train, y_train, cv=5)

In [ ]:
print('Logistic Regression: ', roc_auc_score(y_train, log_reg_pred))
print('KNears Neighbors: ', roc_auc_score(y_train, knears_pred))
print('Support Vector Classifier: ', roc_auc_score(y_train, svc_pred))
print('Decision Tree Classifier: ', roc_auc_score(y_train, tree_pred))

In [ ]:
print(classification_report(y_test,log_reg.predict(x_test)))
print(classification_report(y_test,knears_neighbors.predict(x_test)))
print(classification_report(y_test,svc.predict(x_test)))

In [ ]:
!pip install keras

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

n_inputs =x_train.shape[1]

undersample_model = Sequential([
    Dense(n_inputs, input_shape=(n_inputs, ), activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
])

In [ ]:
undersample_model.summary()

In [ ]:
undersample_model.compile(Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
undersample_model.fit(x_train, y_train, validation_split=0.2, batch_size=25, epochs=20, shuffle=True, verbose=2)

In [ ]:
undersample_model.evaluate(x_test,y_test)